# Test skill

## TEST output player HP

In [1]:
from characters.ken import Ken
import time

# Choose attack mode: 'modern' or 'classic'
ken = Ken(attack_mode='modern')

def test_basic_attack():
    ken.impl.light_attack()     # Perform light attack
    time.sleep(1)
    ken.impl.medium_attack()    # Perform medium attack
    time.sleep(1)
    ken.impl.heavy_attack()     # Perform heavy attack
    time.sleep(1)
    ken.impl.special_attack()   # Perform heavy attack
    time.sleep(1)
    
def test_other_commands():
    ken.impl.drive_parry()      # defense
    time.sleep(1)
    ken.impl.drive_impact()     # Drive Impact
    time.sleep(2)
    ken.impl.throw()            # Throw
    time.sleep(1)
    ken.impl.assist()           # OC
    time.sleep(1)

def test_move_left_right_crouch():
    ken.impl.move_jump()
    time.sleep(3)
    
    # Move Left
    ken.impl.move_left_continuously()   
    time.sleep(3)
    ken.impl.stop_movement()
    time.sleep(3)
    
    # Move Right
    ken.impl.move_right_continuously()
    time.sleep(3)
    ken.impl.stop_movement()
    time.sleep(3)
    
    # crouch
    ken.impl.move_crouch_continuously()
    time.sleep(3)
    ken.impl.stop_movement()
    time.sleep(3)

def test_up_down_left_right():
    # down left
    ken.impl.down_left_continuously()
    time.sleep(2)
    ken.impl.stop_movement()
    time.sleep(2)
    
    # down right
    ken.impl.down_right_continuously()
    time.sleep(2)
    ken.impl.stop_movement()
    time.sleep(2)
    
    # up left
    ken.impl.up_left_continuously()
    time.sleep(2)
    ken.impl.stop_movement()
    
    # up right
    ken.impl.up_right_continuously()
    time.sleep(2)
    ken.impl.stop_movement()

def test_ken_special_skills():
    print("Testing Ken's basic special moves...")

if __name__ == "__main__":
    time.sleep(2)
    ken = Ken(attack_mode='modern')
    for n in range(3):
        ken.thunder_kick()
        time.sleep(1)
        

In [1]:
import cv2
import keyboard
from image_processing.screen_capture import screen_capture
from image_processing.player_hp_detector import L_player_HP, R_player_HP

while True:
    player_blood = screen_capture((115, 85, 1165, 100))
    cv2.imshow("123", player_blood)
    print(f"\rLeft player HP: {L_player_HP(player_blood)}, Right player HP: {R_player_HP(player_blood)}", end='')
    cv2.waitKey(1)
    if keyboard.is_pressed('q'):
        break
cv2.destroyAllWindows()

Left player HP: 52, Right player HP: 49